# Satellite Location Demo

Timeliness is an important aspect for countless uses of Earth Observation (EO) data. Humanitarians and emergency response organizations, for example, benefit from imagery immediately before or after an emergency.

Knowing when an image was captured of an area of interested, or when it will be captured next, can help humanitarians emergency response organizations better understand where aid is most needed.

This notebook demonstrates how the location of EO satellites can be computed in an area of interest. This can be used as a proxy of where images were and will be captured. 

These techniques could be combined with those in other notebooks in this repository, such as loading STAC items and calculating how a disaster impacts people, to create automated analysis for past and future disasters.

## Load Libraries and Data

We'll use the [sgp4](https://pypi.org/project/sgp4/) and [skyfield](https://rhodesmill.org/skyfield/) libraries, which help us calculate the position of satellites using complex orbital physics. Shapely will help us with some of the geometry creation and calculations, and Lonboard will be used to visualize the results.

In [1]:
from skyfield.api import load, wgs84, Timescale

from datetime import datetime, timedelta, timezone

from lonboard import viz, Map, ScatterplotLayer, basemap, PathLayer
from lonboard.colormap import apply_continuous_cmap
from palettable.colorbrewer.sequential import Oranges_9

import geopandas as gpd
import pandas as pd
from shapely import Point, LineString, Polygon

## Calculating the position of a satellite

We'll load information about specific satellites from [Celestrak](https://celestrak.org). Due to the complexities of orbital physics, data about satellites needs to be updated frequently is only accurate for about 1 week before and after "epoch".

For this example, we'll calculate the position of Sentinel 2b.

In [2]:
sentinel_2b_url = "https://celestrak.org/NORAD/elements/gp.php?CATNR=42063"

sentinel_2b = load.tle_file(sentinel_2b_url, reload=True, filename="sentinel_2b")

[#################################] 100% sentinel_2b


In [3]:
satellite = sentinel_2b[0]
satellite

<EarthSatellite SENTINEL-2B catalog #42063 epoch 2024-09-30 04:08:29 UTC>

For sudden-onset emergencies, the time immediately before an after an event are the most critical. We can set a time range of +2 days (48h) from a certain point in time. For the purposes of this notebook, we'll use `now` (the time when the cell is run). These techniques can also be applied in the past, if a before/after analysis is needed.

In [4]:
# Calculate the current UTC time (without microseconds), then creating a time range + 48h

# load timescale
ts = load.timescale()

# load "ephemeris" (https://rhodesmill.org/skyfield/api.html#planetary-ephemerides), which let's us determine if a satellite is illuminated by the sun (as a proxy of if the ground is lit)
eph = load('de421.bsp')

now = datetime.now(timezone.utc).replace(microsecond=0)
t1 = now
t2 = now + timedelta(days=2)

To calculate the position of the satellite throughout the time range, we can initiate a timer and then calculate the position at each time step (defined below).

In [5]:
# initiate

timer = t1

df = pd.DataFrame(columns=['satellite', 'timestamp', 'coordinates', 'lng', 'lat', 'daytime'])

rows = []

The `location_iteration` function lodes the geocentric location, calculates the latitude and longitude, converts them to decimal degrees, and saves them as a point coordinate. It also checks if the satellite is sunlit at the time of calculation, which can be used as an approximation of if it is daytime below.

In [6]:
def location_iteration(timer, sat):
    geocentric = sat.at(Timescale.from_datetime(ts, timer))
    lat, lon = wgs84.latlon_of(geocentric)
    longitude = lon.degrees
    latitude = lat.degrees
    daytime = geocentric.is_sunlit(eph)
    coords = Point(longitude, latitude)

    return timer, longitude, latitude, coords, daytime

Then, we can iterate over the time frame. The more frequent the measurements, the slower the calculation takes.

In [7]:
while timer <= t2:
    timer, longitude, latitude, coords, daytime = location_iteration(timer, sentinel_2b[0])

    row = pd.DataFrame({'satellite': satellite.name, 'timestamp': timer, 'coordinates': [coords], 'lng': longitude, 'lat': latitude, 'daytime': daytime}, index=[0])
    rows.append(row)    

    timer += timedelta(seconds=15) # ~13 seconds

df = pd.concat(rows, ignore_index=True)
df["time_string"] = df["timestamp"].dt.strftime('%Y-%m-%d %X')

We can then save this as a geodataframe.

In [8]:
sentinel_2b_gdf = gpd.GeoDataFrame(df, geometry="coordinates")
sentinel_2b_gdf

,satellite,timestamp,coordinates,lng,lat,daytime,time_string
0,SENTINEL-2B,2024-09-30 08:10:17+00:00,POINT (40.97457 35.03148),40.974574,35.031476,True,2024-09-30 08:10:17
1,SENTINEL-2B,2024-09-30 08:10:32+00:00,POINT (40.71589 34.14938),40.715892,34.149378,True,2024-09-30 08:10:32
2,SENTINEL-2B,2024-09-30 08:10:47+00:00,POINT (40.46124 33.26684),40.461238,33.266844,True,2024-09-30 08:10:47
3,SENTINEL-2B,2024-09-30 08:11:02+00:00,POINT (40.21040 32.38389),40.210402,32.383893,True,2024-09-30 08:11:02
4,SENTINEL-2B,2024-09-30 08:11:17+00:00,POINT (39.96319 31.50054),39.963187,31.500541,True,2024-09-30 08:11:17
...,...,...,...,...,...,...,...
11516,SENTINEL-2B,2024-10-02 08:09:17+00:00,POINT (-144.33830 -3.12798),-144.338298,-3.127984,False,2024-10-02 08:09:17
11517,SENTINEL-2B,2024-10-02 08:09:32+00:00,POINT (-144.53438 -2.23887),-144.534385,-2.238865,False,2024-10-02 08:09:32
11518,SENTINEL-2B,2024-10-02 08:09:47+00:00,POINT (-144.73032 -1.34969),-144.730316,-1.349689,False,2024-10-02 08:09:47
11519,SENTINEL-2B,2024-10-02 08:10:02+00:00,POINT (-144.92616 -0.46047),-144.926156,-0.460466,False,2024-10-02 08:10:02


Let's refine our area of interest to see when/where satellites pass overhead. As the 2024 hurricane season in the Americas is under way, we can focus on the US state of Florida for our analysis.

In [9]:
min_lon = x_min = -89
max_lon = x_max = -74
min_lat = y_min = 22
max_lat = y_max = 32

bbox=[min_lon, max_lon, min_lat, max_lat]
bbox

# Defining our AOI as a polygon shape
florida = Polygon([(x_min, y_min), (x_max, y_min), (x_max, y_max), (x_min, y_max), (x_min,y_min)])

In [10]:
mask_lon = (sentinel_2b_gdf.lng >= min_lon) & (sentinel_2b_gdf.lng <= max_lon)
mask_lat = (sentinel_2b_gdf.lat >= min_lat) & (sentinel_2b_gdf.lat <= max_lat)

aoi = sentinel_2b_gdf.where(mask_lon & mask_lat).dropna()

sentinel_2b_daylit = aoi[aoi["daytime"] == True]
sentinel_2b_daylit

,satellite,timestamp,coordinates,lng,lat,daytime,time_string
2018,SENTINEL-2B,2024-09-30 16:34:47+00:00,POINT (-85.91352 31.49217),-85.913522,31.492168,True,2024-09-30 16:34:47
2019,SENTINEL-2B,2024-09-30 16:35:02+00:00,POINT (-86.15727 30.60843),-86.157272,30.608431,True,2024-09-30 16:35:02
2020,SENTINEL-2B,2024-09-30 16:35:17+00:00,POINT (-86.39777 29.72433),-86.397767,29.724326,True,2024-09-30 16:35:17
2021,SENTINEL-2B,2024-09-30 16:35:32+00:00,POINT (-86.63517 28.83987),-86.635175,28.839870,True,2024-09-30 16:35:32
2022,SENTINEL-2B,2024-09-30 16:35:47+00:00,POINT (-86.86966 27.95508),-86.869656,27.955076,True,2024-09-30 16:35:47
2023,SENTINEL-2B,2024-09-30 16:36:02+00:00,POINT (-87.10136 27.06996),-87.101362,27.069960,True,2024-09-30 16:36:02
2024,SENTINEL-2B,2024-09-30 16:36:17+00:00,POINT (-87.33044 26.18454),-87.330437,26.184536,True,2024-09-30 16:36:17
2025,SENTINEL-2B,2024-09-30 16:36:32+00:00,POINT (-87.55702 25.29882),-87.557019,25.298817,True,2024-09-30 16:36:32
2026,SENTINEL-2B,2024-09-30 16:36:47+00:00,POINT (-87.78124 24.41282),-87.781238,24.412816,True,2024-09-30 16:36:47
2027,SENTINEL-2B,2024-09-30 16:37:02+00:00,POINT (-88.00322 23.52655),-88.003220,23.526547,True,2024-09-30 16:37:02


Next, we can visualize these points on a map. They are color coded using a continuous color scale, with white being `now` and darker purple being further after `now`.

In [11]:
# This creates a range from 0-1 to define our colormap.
time_norm = (sentinel_2b_daylit.timestamp - t1) / (t2 - t1)

colors = apply_continuous_cmap(time_norm, Oranges_9)

In [12]:
layer = ScatterplotLayer.from_geopandas(
    sentinel_2b_daylit,
    # extensions=[filter_extension],
    get_fill_color=colors,
    radius_min_pixels = 3
    # get_filter_value=filter_values,
    # filter_range=initial_filter_range,
)

m = Map(
    layer,
    basemap_style = basemap.CartoBasemap.DarkMatter,
    )
m

/opt/homebrew/lib/python3.10/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

We can clearly see the passes that Sentinel 2b takes in our area of interest and during our time frame, and during daytime hours.

---

## Calculating the position of a satellite constellation

Usually, humanitarians or emergency response organizations aren't concerned about which satellite captures an image, as long as they have a timely image with the appropriate resolution, (lack of) cloud cover, bands, etc.

The following example calculates the position of Planet satellites. With their portfolio's high spatial and temporal resolution, they are able to capture high-resolution images immediately before or after an event.

In [13]:
planet_url = "https://celestrak.org/NORAD/elements/gp.php?GROUP=planet&FORMAT=tle"

planet_swarm = load.tle_file(planet_url, reload=True, filename="planet_swarm")
planet_swarm[0]

[#################################] 100% planet_swarm


<EarthSatellite SKYSAT-A catalog #39418 epoch 2024-09-29 21:20:16 UTC>

We then calculate the locations for all Planet satellites during our time frame in set increments.

In [14]:
timer = t1

rows = []

while timer <= t2:
    for sat in planet_swarm:
        timer, longitude, latitude, coords, daytime = location_iteration(timer, sat)

        row = pd.DataFrame({'satellite': sat.name, 'timestamp': timer, 'coordinates': [coords], 'lng': longitude, 'lat': latitude, 'daytime': daytime}, index=[0])
        rows.append(row)    

    timer += timedelta(minutes=2) # ~ 3.5 min

planet_swarm_df = pd.concat(rows, ignore_index=True)
planet_swarm_df["time_string"] = planet_swarm_df["timestamp"].dt.strftime('%Y-%m-%d %X')


In [15]:
planet_swarm_gdf = gpd.GeoDataFrame(planet_swarm_df, geometry="coordinates")

Again, we want to look at the satellites in our AOI. Because Planet satellites capture optical images, the following shows the next passes that occur when the satellite is lit (as a proxy of when the ground below is lit).

In [16]:
mask_lon = (planet_swarm_gdf.lng >= min_lon) & (planet_swarm_gdf.lng <= max_lon)
mask_lat = (planet_swarm_gdf.lat >= min_lat) & (planet_swarm_gdf.lat <= max_lat)

aoi_planet = planet_swarm_gdf.where(mask_lon & mask_lat).dropna()
aoi_planet_day = aoi_planet[aoi_planet.daytime == True]
aoi_planet_day[aoi_planet_day["timestamp"] > now]

planet_day = planet_swarm_gdf[planet_swarm_gdf.daytime == True]
planet_day[planet_day["timestamp"] > now]

,satellite,timestamp,coordinates,lng,lat,daytime,time_string
123,SKYSAT-A,2024-09-30 08:12:17+00:00,POINT (-144.79334 -65.13927),-144.793337,-65.139274,True,2024-09-30 08:12:17
124,SKYSAT-B,2024-09-30 08:12:17+00:00,POINT (53.76233 27.52190),53.762327,27.521900,True,2024-09-30 08:12:17
127,SKYSAT-C5,2024-09-30 08:12:17+00:00,POINT (19.37024 37.43141),19.370241,37.431412,True,2024-09-30 08:12:17
129,SKYSAT-C3,2024-09-30 08:12:17+00:00,POINT (13.91201 19.73941),13.912006,19.739405,True,2024-09-30 08:12:17
130,SKYSAT-C11,2024-09-30 08:12:17+00:00,POINT (90.92110 14.82834),90.921099,14.828342,True,2024-09-30 08:12:17
...,...,...,...,...,...,...,...
177221,FLOCK 4BE-32,2024-10-02 08:10:17+00:00,POINT (-124.77185 -67.41858),-124.771852,-67.418579,True,2024-10-02 08:10:17
177222,FLOCK 4BE-30,2024-10-02 08:10:17+00:00,POINT (-106.53382 -77.64441),-106.533820,-77.644415,True,2024-10-02 08:10:17
177223,FLOCK 4BE-11,2024-10-02 08:10:17+00:00,POINT (-118.13426 -72.82073),-118.134259,-72.820732,True,2024-10-02 08:10:17
177224,FLOCK 4BE-33,2024-10-02 08:10:17+00:00,POINT (-126.58304 -65.25726),-126.583040,-65.257257,True,2024-10-02 08:10:17


## Visualize path of satellites

Instead of visualizing points on a map, a better representation would be a line. We can "connect the dots" calculated in the previous step.

In [17]:
def create_individual_linestrings(coords, timestamps, time_strings):
    coords_list = coords.tolist()
    timestamps_list = timestamps.tolist()
    time_strings_list = time_strings.tolist()
    line_segments = [
        {'linestring': LineString([coords_list[i], coords_list[i + 1]]), 'timestamp': timestamps_list[i], 'time_string': time_strings_list[i]} 
        for i in range(len(coords_list) - 1)
    ]
    return line_segments

In [18]:
new_rows = []

for satellite, group in planet_day.groupby('satellite'):
    line_segments = create_individual_linestrings(group['coordinates'], group['timestamp'], group['time_string'])
    for segment in line_segments:
        new_rows.append({'satellite': satellite, 'linestring': segment['linestring'], 'timestamp': segment['timestamp'], 'time_string': segment['time_string']})

path_segments = gpd.GeoDataFrame(new_rows, geometry='linestring')
path_segments

,satellite,linestring,timestamp,time_string
0,FLOCK 4BE-1,"LINESTRING (-56.25717 -82.58685, -100.74972 -7...",2024-09-30 08:10:17+00:00,2024-09-30 08:10:17
1,FLOCK 4BE-1,"LINESTRING (-100.74972 -79.14175, -118.96363 -...",2024-09-30 08:12:17+00:00,2024-09-30 08:12:17
2,FLOCK 4BE-1,"LINESTRING (-118.96363 -72.85942, -127.52718 -...",2024-09-30 08:14:17+00:00,2024-09-30 08:14:17
3,FLOCK 4BE-1,"LINESTRING (-127.52718 -65.85457, -172.98478 6...",2024-09-30 08:16:17+00:00,2024-09-30 08:16:17
4,FLOCK 4BE-1,"LINESTRING (-172.98478 68.84880, 175.55200 75....",2024-09-30 08:52:17+00:00,2024-09-30 08:52:17
...,...,...,...,...
112362,TANAGER-1,"LINESTRING (18.87117 -71.40974, 3.70230 -77.92...",2024-10-02 07:52:17+00:00,2024-10-02 07:52:17
112363,TANAGER-1,"LINESTRING (3.70230 -77.92690, -33.98769 -82.3...",2024-10-02 07:54:17+00:00,2024-10-02 07:54:17
112364,TANAGER-1,"LINESTRING (-33.98769 -82.32836, -86.68870 -80...",2024-10-02 07:56:17+00:00,2024-10-02 07:56:17
112365,TANAGER-1,"LINESTRING (-86.68870 -80.79299, -111.33058 -7...",2024-10-02 07:58:17+00:00,2024-10-02 07:58:17


If these line segments are not within the AOI, we can drop them from the dataframe.

In [19]:
def is_within_aoi(linestring, florida):
    return linestring.intersects(florida)

In [20]:
path_segments['within_aoi'] = path_segments['linestring'].apply(lambda x: is_within_aoi(x, florida))
path_segments_clipped = path_segments[path_segments['within_aoi']].drop(columns='within_aoi')

path_segments_clipped['length'] = path_segments_clipped['linestring'].apply(lambda x: x.length)

# This is a way to drop segments that are not proper representations of paths, such as errors caused by traversing the International Date Line, or jumps between calculated points when a satellite enters and exits a nighttime area.
path_segments_clipped = path_segments_clipped[path_segments_clipped["length"] <= 75].drop(columns='length')

As with the single satellite, we can create a color scale that shows the time of the satellite's pass for each line segment. In this example, white is closer to `now` and orange is closer to +48h.

In [21]:
# This creates a range from 0-1 to define our colormap.
time_norm_planet_swarm = (path_segments_clipped.timestamp - t1) / (t2 - t1)

colors_planet_swarm = apply_continuous_cmap(time_norm_planet_swarm, Oranges_9, alpha=.5)

We can roughly estimate the width of the ground coverage of a Planet satellite to be 6km (i.e., the width of an image is 6km). To visualize this, we can set `get_width` to be 6km.

In [22]:
layer2 = PathLayer.from_geopandas(
    path_segments_clipped,
    get_color = colors_planet_swarm,
    get_width=6000,
    opacity=1,
    auto_highlight=True
)

/opt/homebrew/lib/python3.10/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


In [23]:
m2 = Map(
    [layer2],
    basemap_style = basemap.CartoBasemap.DarkMatter,
    )
m2

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

As expected, there are many more passes than by a single satellite. Due to small area of interests for many events, cloud cover, and other variables, a larger constellation lets practitioners then narrow down their search.

The visualization of these paths give practitioners a good sense of the time an image of an area of interest will be captured. For example, a practitioner would be interested in an AOI of this size, but might be particularly interested in images of Miami. They could zoom in to Miami and see if a path covers or comes close to Miami. If so, and if there isn't excessive cloud cover, an image could be made available or tasked. If multiple paths cover Miami, that indicates a better chance of capturing a valuable image.

Once a satellite is calculated to have passed over an AOI, the next step could be to analyze the imagery, ideally using the other STAC and COG focused notebooks in this repository. Combining a positioning calculation with automated image loading and processing could be the foundations for a powerful EO-based monitoring and alerting system.